# Запуск имитационной модели системы массового обслуживания без приоритетов

Имитационная модель (ИМ) СМО поддерживает различные типы распределений входного потока и обслуживания, произвольное число каналов. Возможно задание максимальной длины очереди. По умолчанию - бесконечная очередь. В нотации Кендалла: GI/G/n/r и GI/G/n

Для запуска ИМ требуется 4 шага:
- создание экземпляра ИМ
- задание входного потока
- задание распределения обслуживания
- запуск модели


#### Импорт класса QueueingSystemSimulator и создание экземпляра ИМ.
При создании укажем число каналов *n* и максимальную длину очереди *r*. Если не указать, то по умолчанию очередь считается неограниченной.

In [20]:
from most_queue.sim.qs_sim import QueueingSystemSimulator

n = 3
r = 100

qs = QueueingSystemSimulator(n, buffer=r)

#### Задаем входной поток.
Метод *set_sources* принимает два параметра:
- параметры распределения
- тип распределения

Для экспоненциального достаточно передать один параметр - интенсивность входящего потока. Для большинства распределений параметры задаются списком

#### Поддерживаемые виды распределений вх потока
| Вид распределения | Тип | Параметры |
| --- | --- | --- |
| Экспоненциальное | 'М' | mu
| Гиперэкспоненциальное 2-го порядка | 'Н' | [y1, mu1, mu2]
| Гамма-распределение | 'Gamma' | [mu, alpha]
| Эрланга | 'E'| [r, mu]
| Кокса 2-го порядка | 'C' | [y1, mu1, mu2]
| Парето | 'Pa'| [alpha, K]
| Детерминированное | 'D' | [b]
| Равномерное | 'Uniform'| [mean, half_interval]
| Нормальное | 'Norm'| [mean, standard_deviation]

In [21]:
l = 1.0
qs.set_sources(l, 'M')

#### Задаем распределение времени обслуживания. Рассчитаем интенсивность обслуживания через коэффициент загрузки и число каналов

In [22]:
ro = 0.8  # коэффициент загрузки
mu = l / (ro * n)  # интенсивность обслуживания
qs.set_servers(mu, 'M')

#### Для запуска ИМ необходимо вызвать метод *run* и передать число заявок

In [23]:
num_of_jobs = 1000000
qs.run(num_of_jobs)

Start simulation



100%|██████████| 1000000/1000000 [00:03<00:00, 278094.36it/s]


Simulation is finished



#### После окончания ИМ можно получить начальные моменты времени ожидания и пребывания, а также вероятности состояний системы

In [24]:
print("Time spent ", qs.time_spent)
v_sim = qs.v
w_sim = qs.w
p_sim = qs.p
print(v_sim)

Time spent  1.453125
[5.003024348895862, 44.626632671328096, 561.7197477712853]


#### Сверим результаты ИМ с результатами численного расчета для модели M/M/n/r

In [25]:
from most_queue.theory import mmnr_calc
from most_queue.general_utils.tables import times_print, probs_print

w = mmnr_calc.MMnr_calc.get_w(l, mu, n, r)
times_print(w_sim, w)

Начальные моменты времени ожидания заявок в системе:
   № момента   |     Числ      |      ИМ       
---------------------------------------------
       1       |    2.5888     |     2.603     
       2       |     20.71     |    20.607     
       3       |    248.52     |    240.52     



#### Проверим корректность ИМ с помощью результатов расчета M/D/n

In [26]:
from most_queue.theory import m_d_n_calc

qs = QueueingSystemSimulator(n)

qs.set_sources(l, 'M')
qs.set_servers(1.0 / mu, 'D')

qs.run(num_of_jobs)

mdn = m_d_n_calc.M_D_n(l, 1 / mu, n)
p_ch = mdn.calc_p()
p_im = qs.get_p()

print("Time spent ", qs.time_spent)

probs_print(p_im, p_ch, 10)

Start simulation



100%|██████████| 1000000/1000000 [00:03<00:00, 324213.98it/s]


Simulation is finished

Time spent  0.953125
------------------------------------
       Вероятности состояний        
------------------------------------
 №  |     Числ      |      ИМ       
------------------------------------
 0  |   0.049841    |   0.049209    
 1  |    0.13286    |    0.13181    
 2  |    0.18476    |    0.18292    
 3  |    0.18195    |    0.18091    
 4  |    0.14595    |    0.14683    
 5  |    0.10403    |    0.10489    
 6  |   0.069852    |   0.070424    
 7  |   0.045764    |   0.046102    
 8  |   0.029764    |   0.030399    
 9  |   0.019336    |   0.019967    
------------------------------------

